In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from statsmodels.sandbox.regression.predstd import wls_prediction_std

plt.rcParams.update({'font.size': 14})

## Laborator: exerciţii şi probleme

__Q1__. Pentru swiss dataset, realizați modelul de regresie liniară pentru Fertility în funcție de Agriculture și Education. Remarcați intercept și cei doi coeficienți.

Mai departe arătați cum coeficientul regresorului $x_1$ (Agriculture) este de fapt coeficientul regresiei caracteristice răspunsului $y$ (Fertility) și regresorului $x_2$ (Education) din care contribuția celuilalt regresor ($x_1$) a fost deja înlăturată.

__Hint1__: contribuția unui regresor nu mai este prezentă în reziduul asociat, și se înlătură folosind regresia liniară (revedeți slide-urile 42-end din cursul 7).

__Hint2__: folosiți proprietatea .resid a modelului deja potrivit.

In [ ]:
swiss = pd.read_csv('swiss.csv')
swiss.columns = ['District', 'Fertility', 'Agriculture', 'Examination', 'Education', 'Catholic', 
                 'InfantMortality']
swiss.head()

In [ ]:
lm_a = smf.ols(formula='Fertility ~ Agriculture + Education', data=swiss).fit()
lm_a.summary()

In [ ]:
print(f'R Q1. a)')

intercept = lm_a.params[0]
print(f'Intercept: {intercept}')

print(f'Coeficient Agriculture: {lm_a.params[1]}')
print(f'Coeficient Education: {lm_a.params[2]}')

In [ ]:
x1 = 'Agriculture'
x2 = 'Education'
y = 'Fertility'

lm_x1 = smf.ols(formula='Agriculture ~ Education', data=swiss).fit()
print('Agriculture ~ Education')
print(lm_x1.params)

lm_x2 = smf.ols(formula='Fertility ~ Education', data=swiss).fit()
print('Fertility ~ Education')
print(lm_x2.params)

In [ ]:
print(f'R Q1. b)')

swiss['residAgriculture'] = lm_x1.resid
swiss['residFertility'] = lm_x2.resid

lm = smf.ols(formula='residFertility ~ residAgriculture - 1', data=swiss).fit()
lm.summary()

In [ ]:
print(f'R Q1. b)')

lm = smf.ols(formula='Fertility ~ Agriculture + Education', data=swiss).fit()
lm.summary()

__Q2__. Pentru setul mtcars, considerați variabila categorială 'număr de cilindri'.

a) Ridicați diagrama pair plot.

b) Calculați coeficienții de regresie. Există variabile care par să explice consumul?

c) Ridicați pe un scatter plot regresia mpg funcție de horsepower.

d) În funcție și de numărul de cilindri, realizați două linii de regresie dacă presupunem că nu există interacțiune între horsepower și numărul de cilindri.

e) În funcție și de numărul de cilindri, realizați două linii de regresie dacă presupunem acum că există totuși interacțiune.

In [ ]:
mtcars = pd.read_csv('mtcars.csv')
mtcars.head()

In [ ]:
mtcars.describe()

In [ ]:
print(f'R Q1. a) Pair plot pentru "cyl".')

cyl = mtcars['cyl']

sns.set(style='ticks') #trage linii de la cifrele de sub grafic catre grafic
sns.pairplot(mtcars, hue='cyl', kind='reg')
plt.show()

In [ ]:
print(f'R Q1. b) Coeficientii de regresie "cyl".')
print()

lm = smf.ols(formula='cyl ~ mpg + disp + hp + drat + wt + qsec + vs + am + gear + carb', 
             data=mtcars).fit()
print(lm.params)
lm.summary()

In [ ]:
print(f'R Q1. b) Coeficientii de regresie "mpg".')
print()

lm = smf.ols(formula='mpg ~ cyl + disp + hp + drat + wt + qsec + vs + am + gear + carb', 
             data=mtcars).fit()
print(lm.params)
lm.summary()

In [ ]:
print(f'R Q1. b) Consumul este explicat de variabila "wt" si "am".')

In [ ]:
lm = smf.ols(formula='mpg ~ hp', data=mtcars).fit()
print(lm.params)
lm.summary()

In [ ]:
print(f'R Q1. c) Scatter plot "mpg" in functie de horsepower.')

hp = mtcars['hp'].values #X
mpg = mtcars['mpg'].values #Y

beta0 = lm.params[0] #lm 'mpg ~ hp'
beta1 = lm.params[1]

hp_mean = np.mean(hp)

x1 = np.linspace(np.min(hp), np.max(hp), 100)
y1 = beta0 + beta1 * x1

fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.scatter(hp, mpg, c='teal', alpha=0.5, s=50)
ax.plot(x1, y1, lw=3, c='teal')
ax.set(xlabel='hp', ylabel='mpg')
ax.grid(True)
plt.show()

In [ ]:
print(f'R Q1. d) Lipsa interactiune intre horsepower si numarul de cilindri.')

params = smf.ols(formula='mpg ~ hp + cyl', data=mtcars).fit().params

x = hp #determinat mai sus din mtcars.csv
fig, ax = plt.subplots(1, 1, figsize=(15, 5))

c = mtcars['cyl'].copy()
c[mtcars['cyl'] == 4] = 'Red'
c[mtcars['cyl'] == 6] = 'Green'
c[mtcars['cyl'] == 8] = 'Blue'
ax.scatter(hp, mpg, c=c)

ax.plot(x, params[0] + params[1] * x + params[2] * 4, 'r')
ax.plot(x, params[0] + params[1] * x + params[2] * 6, 'g')
ax.plot(x, params[0] + params[1] * x + params[2] * 8, 'b')
ax.set_xlabel('hp')
ax.set_ylabel('mpg')
ax.legend(['4 cylinders', '6 cylinders', '8 cylinders'])
plt.grid()
plt.show()

In [ ]:
print(f'R Q1. e) Exista interactiune intre horsepower si numarul de cilindri.')

formula = 'mpg ~ hp * C(cyl)'

params = smf.ols(formula=formula, data=mtcars).fit().params

x = hp #determinat mai sus din mtcars.csv
fig, ax = plt.subplots(1, 1, figsize=(15, 5))

c = mtcars['cyl'].copy()
c[mtcars['cyl'] == 4] = 'Red'
c[mtcars['cyl'] == 6] = 'Green'
c[mtcars['cyl'] == 8] = 'Blue'
ax.scatter(hp, mpg, c=c)

ax.plot(x, params[0] + params[1] * x + (params[2] + params[3]) * x * 4, 'r')
ax.plot(x, params[0] + params[1] * x + (params[2] + params[3]) * x * 6, 'g')
ax.plot(x, params[0] + params[1] * x + (params[2] + params[3]) * x * 8, 'b')
ax.set_xlabel('hp')
ax.set_ylabel('mpg')
ax.legend(['4 cylinders', '6 cylinders', '8 cylinders'])
plt.grid()
plt.show()

In [ ]:
lm = smf.ols(formula=formula, data=mtcars).fit() #lm 'mpg ~ hp * C(cyl)'
print(lm.params)
lm.summary()